# Fine-Tune Alpaca + Unsloth Model on WhatApp data

Table of Contents:
*   Install dependencies
*   Preprocess whatsapp chat data
*   Train the model
*   Inference
*   Saving, loading finetuned models
    * Unsloth method
    * Merge model with LoRA weights
    * Push model to github


Based on Unsloth Ai <a href="https://github.com/unslothai/unsloth"> Github </a>

## Install dependencies

In [ ]:
!pip install datasets

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    #token = "", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Preprocess whatsapp chat data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv

# Function to extract messages and users from input file
def extract_messages(input_file):
    messages = []
    users = []
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            parts = line.split('-', 1)
            if len(parts) > 1:
                user_message = parts[1].strip()
                user_message_parts = user_message.split(':', 1)
                if len(user_message_parts) > 1:
                    user = user_message_parts[0].strip()
                    text = user_message_parts[1].strip()
                    users.append(user)
                    messages.append(text)
    return users, messages

# Function to format conversation data with dynamic user inputs
def format_chat_template(users, messages, question_user, response_user):
    convo_data = []
    i = 0
    while i < len(messages):
        questions = []
        while i < len(messages) and users[i] == question_user:
            questions.append(messages[i])
            i += 1
        question = "\n".join(questions)

        responses = []
        while i < len(messages) and users[i] == response_user:
            responses.append(messages[i])
            i += 1
        response = "\n".join(responses)

        convo_data.append({
            'question': question,
            'response': response
        })
    return convo_data

# Function to write conversation data to a CSV file
def write_convo_to_csv(convo_data, output_file):
    with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['question', 'response'])
        for convo in convo_data:
            csv_writer.writerow([convo['question'], convo['response']])

# Main function to combine all steps
def main(input_file, output_file, question_user, response_user):
    users, messages = extract_messages(input_file)
    convo_data = format_chat_template(users, messages, question_user, response_user)
    write_convo_to_csv(convo_data, output_file)
    print(f"Messages extracted, formatted, and stored in {output_file}")

# Specify the input and output file names
input_file = '/content/drive/MyDrive/WhatsApp_Chat.txt'
output_file = '/content/drive/MyDrive/output.csv'

# Define the user names for questions and responses
question_user = "Test"
response_user = "User"

# Run the main function
main(input_file, output_file, question_user, response_user)

Messages extracted, formatted, and stored in /content/drive/MyDrive/output.csv


## Train the model

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = "Generate a response to the following question:"
    inputs       = examples["question"]
    outputs      = examples["response"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/output.csv", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,batch_size=32)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/5 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.156300
2,0.707400
3,1.184600
4,0.904200
5,0.502600
6,0.859600
7,0.627000
8,0.142200
9,0.346300
10,0.224400


## Inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        "Hello", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

## Saving, loading finetuned models

### Unsloth method

In [ ]:
output_dir = "/content/drive/MyDrive/AI_models/unsloth/finetuned"

In [ ]:
model.save_pretrained(output_dir) # Local saving
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/AI_models/unsloth/finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/AI_models/unsloth/finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/AI_models/unsloth/finetuned/tokenizer.model',
 '/content/drive/MyDrive/AI_models/unsloth/finetuned/added_tokens.json',
 '/content/drive/MyDrive/AI_models/unsloth/finetuned/tokenizer.json')

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = output_dir, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        "who's going on the movie ?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

### Merge Lora and base model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
    #token = "",
    #load_in_4bit=True,
)
model = PeftModel.from_pretrained(base_model, output_dir)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", trust_remote_code=True , token="")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
inputs = tokenizer(
"like to find out the full day meeting package rate?",
     return_tensors = "pt")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

### Push to HuggingFace

In [ ]:
!huggingface-cli login

In [ ]:
model.push_to_hub(output_dir, max_shard_size='2GB')
tokenizer.push_to_hub(output_dir)